In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

import numpy as np 
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train_clever.csv to train_clever.csv


In [4]:
from google.colab import files
uploaded = files.upload()

Saving test_clever.csv to test_clever.csv


In [92]:
train = pd.read_csv("train_clever.csv")
train

,ID,Answer,Question
0,16019,0,Крупнейшим производителем кукурузы является:
1,5121,0,Кто первый в РОССИЮ завёз картошку ?
2,17169,0,Кто из мореплавателей сделал первое кругосветн...
3,15480,0,Как зовут героя серии романов Бариса Акунина?
4,3312,0,Какая сборная выйграла чемпионат мира по футбо...
...,...,...,...
19995,14210,0,В каком году изобрели компьютерную мышь?
19996,17590,0,Каким прозвищем удостоен известный американски...
19997,20342,0,Дата начала Второй Великой Отечественной войны
19998,9183,0,В какой Республике находиться город Стерлитамак


In [93]:
test = pd.read_csv("test_clever.csv")
test

,ID,Question
0,11083,В каком году основан Санкт-Петербург?
1,18349,Какой город Золотого кольца России-город худож...
2,11374,Каким цветом написана надпись магазина «пятеро...
3,9699,Сколько весит пирамида Хеопса?
4,17748,В какой стране киберспорт впервые признали вид...
...,...,...
9995,28782,В каком году началась Первая Мировая война?
9996,13015,Самое популярное женское имя
9997,26647,Сколько на данный момент снято официальных фил...
9998,9578,У какого айфона появился впервые появился Touc...


In [94]:
# Посмотрим размерность данных
print(train.shape)
print(test.shape)

(20000, 3)
(10000, 2)


In [95]:
train.isna().sum()

ID          0
Answer      0
Question    0
dtype: int64

In [96]:
train['Answer'].value_counts()

0    17885
1     2115
Name: Answer, dtype: int64

In [97]:
train['Question'] = train['Question'].str.replace("\r", " ")
train['Question'] = train['Question'].str.replace("\n", " ")
train['Question'] = train['Question'].str.replace("    ", " ")

In [98]:
train['Question'] = train['Question'].str.replace('"', '')
list("?:!.,;")
punctuation_signs = list("?:!.,;")

for punct_sign in punctuation_signs:
    train['Question'] = train['Question'].str.replace(punct_sign, '')

In [99]:
train['Question'] = train['Question'].str.lower() 
test['Question'] = test['Question'].str.lower()

In [100]:
test['Question'] = test['Question'].str.replace("\r", " ")
test['Question'] = test['Question'].str.replace("\n", " ")
test['Question'] = test['Question'].str.replace("    ", " ")
test['Question'] = test['Question'].str.replace('"', '')
list("?:!.,;")
punctuation_signs = list("?:!.,;")


for punct_sign in punctuation_signs:
    test['Question'] = test['Question'].str.replace(punct_sign, '')

In [101]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [102]:
lemmatized_text = []
for i in range(len(train)):
    lemmatized_words = []
    words = train.loc[i]['Question'].split(' ')
    for word in words:
        lemma = morph.parse(word)
        lemmatized_words.append(lemma[0].normal_form)
    text = " ".join(lemmatized_words)
    lemmatized_text.append(text)
train['Question'] = lemmatized_text

In [103]:
train

,ID,Answer,Question
0,16019,0,крупный производитель кукуруза являться
1,5121,0,кто первый в россия завезти картошка
2,17169,0,кто из мореплаватель сделать первый кругосветн...
3,15480,0,как звать герой серия роман барис акунин
4,3312,0,какой сборная выйграло чемпионат мир по футбол...
...,...,...,...
19995,14210,0,в какой год изобрести компьютерный мышь
19996,17590,0,какой прозвище удостоить известный американски...
19997,20342,0,дата начало второй великий отечественный война
19998,9183,0,в какой республика находиться город стерлитамак


In [104]:
lemmatized_text = []
for i in range(len(test)):
    lemmatized_words = []
    words = test.loc[i]['Question'].split(' ')
    for word in words:
        lemma = morph.parse(word)
        lemmatized_words.append(lemma[0].normal_form)
    text = " ".join(lemmatized_words)
    lemmatized_text.append(text)
test['Question'] = lemmatized_text

In [105]:
test

,ID,Question
0,11083,в какой год основать санкт-петербург
1,18349,какой город золотой кольцо россии-город художник
2,11374,какой цвет написать надпись магазин «пятерочка»
3,9699,сколько весить пирамида хеопёс
4,17748,в какой страна киберспорт впервые признать вид...
...,...,...
9995,28782,в какой год начаться первый мировой война
9996,13015,самый популярный женский имя
9997,26647,сколько на данный момент снятой официальный фи...
9998,9578,у какой айфон появиться впервые появиться touc...


In [106]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

In [107]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(train['Question'], train['Answer'], test_size=0.2, random_state=42)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts = count_vectorizer.transform(test["Question"])


clf = LogisticRegression(random_state=42)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [108]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None, average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')

    roc_auc = roc_auc_score(y_test, y_predicted_counts)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)

    return accuracy, precision, recall, f1, roc_auc

accuracy, precision, recall, f1, roc_auc = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f, roc_auc = %.3f" % (accuracy, precision, recall, f1, roc_auc))

accuracy = 0.891, precision = 0.822, recall = 0.891, f1 = 0.852, roc_auc = 0.500


In [109]:
y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)

y_test_pred

,index,Answer
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,0


In [110]:
y_test_pred.to_csv('Clever_Goncharenko.csv', index=False)
files.download("Clever_Goncharenko.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>